### SENTIMENT IDENTIFICATION

1. Human Ratings Task:
a) Get 3 classmates (opinion holders) to write three different opinions about their phone.
b) Get 3 different people (raters) to rate these comments as positive, negative, neutral or can’t-say
c) Take this 3 x 3 matrix and find the inter-rater reliability between your 3 raters using Kappa
d) If you wanted to get the correlation between raters (using Pearson’s rho) what would you do? Then do this.

In [12]:
# Opinions:
#1. It's expensive and the battery doesn't last as long as I'd like, but that hasn't stopped me from thoroughly
    #enjoying my time with the Pixel 4 XL. This is a phone where the overall experience is greater than the sum of its parts, and at the end of the day, it's the Android handset I most eagerly reach for.
#2. Awful. After less than six months the screen is not working. It not original I guess.
#3. I was very excited when i received my phone. Fast processing, however after about two months of using, 
    #white lines appeared on the screen which then cost me additional money to replace the screens.

In [13]:
#Rating are as follows: 
    # + -> +1
    # - -> 0
    # N -> -1

In [14]:
from sklearn.metrics import cohen_kappa_score

rater1 = [1,0,-1]
rater2 = [-1,0,-1]
rater3 = [1,0,-1]

print('Inter-Rater Reliability between Rater1 and Rater2: ',cohen_kappa_score(rater1, rater2))
print('Inter-Rater Reliability between Rater1 and Rater3: ',cohen_kappa_score(rater1, rater3))
print('Inter-Rater Reliability between Rater2 and Rater3: ',cohen_kappa_score(rater2, rater3))

Inter-Rater Reliability between Rater1 and Rater2:  0.4999999999999999
Inter-Rater Reliability between Rater1 and Rater3:  1.0
Inter-Rater Reliability between Rater2 and Rater3:  0.5


In [15]:
import numpy as np

def pearson_correlation(numbers_x, numbers_y):
    x = numbers_x - numbers_x.mean()
    y = numbers_y - numbers_y.mean()
    return (x * y).sum() / np.sqrt((x**2).sum() * (y**2).sum())

In [16]:
r1=np.array(rater1)
r2=np.array(rater2)
r3=np.array(rater3)

In [17]:
print(r1,r2,r3)

[ 1  0 -1] [-1  0 -1] [ 1  0 -1]


In [18]:
print('Pearson\'s rho between Rater1 and Rater2',pearson_correlation(r1,r2))
print('Pearson\'s rho between Rater1 and Rater3',pearson_correlation(r1,r3))
print('Pearson\'s rho between Rater2 and Rater3',pearson_correlation(r2,r3))

Pearson's rho between Rater1 and Rater2 0.0
Pearson's rho between Rater1 and Rater3 1.0
Pearson's rho between Rater2 and Rater3 0.0


In [19]:
print('Pearson\'s rho between Rater2 and Rater3',pearson_correlation(r2,r2))

Pearson's rho between Rater2 and Rater3 1.0


### 3 Bromberg’s Sentiment Program:
Have a look at the simple program that does sentiment analysis. So, take a look at the program and see what is
happening in the different variables, but adding print statements on its variables.
a) Now consider ways to improve the training. Eg if you removed stopwords from the inputs what do you think might
happen?
b) Implement this or another solution in the program and report what happens to the precision and recall of the
classifier.

In [44]:
# http://andybromberg.com/sentiment-analysis-python/
# Andy Bromberg's Simple Sentiment Analysis System
# Uses data from Pang & Lee (2005)
# Uses a Naive Bayes Classifier Train the System
#  NB Updated 2016 for package changes around scores

import re, math, collections, itertools, sys, os
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures, scores
from nltk.probability import FreqDist, ConditionalFreqDist

def evaluate_features(feature_select):
    #reading pre-labeled input and splitting into lines
    #Opens the positive,negative files.
    negSentences = open('rt-polarity-neg.txt', 'r', encoding='utf8')
    posSentences = open('rt-polarity-pos.txt', 'r', encoding='utf8')
    
    #Split the file content by newline character.
    negSentences = re.split(r'\n', negSentences.read())
    #print(negSentences)
    posSentences = re.split(r'\n', posSentences.read())
    #print(posSentences)
    
    posFeatures = []
    negFeatures = []
    # breaks up the sentences into lists of individual words
    # creates instance structures for classifier
    
    for i in posSentences:
        posWords = re.findall(r"[\w']+|[.,!?;]", i)
        posWords = [feature_select(posWords), 'pos']
        posFeatures.append(posWords)
    for i in negSentences:
        negWords = re.findall(r"[\w']+|[.,!?;]", i)
        negWords = [feature_select(negWords), 'neg']
        negFeatures.append(negWords)
        
    posCutoff = int(math.floor(len(posFeatures)*3/4))
    negCutoff = int(math.floor(len(negFeatures)*3/4))
    trainFeatures = posFeatures[:posCutoff] + negFeatures[:negCutoff]
    testFeatures = posFeatures[posCutoff:] + negFeatures[negCutoff:]
    #Runs the classifier on the testFeatures
    classifier = NaiveBayesClassifier.train(trainFeatures)
    
    #Sets up labels to look at output
    referenceSets = collections.defaultdict(set)
    testSets = collections.defaultdict(set)
    for i, (features, label) in enumerate(testFeatures): # enumerate adds number-count to each item
        referenceSets[label].add(i)               # recorded polarity for these test sentences
        predicted = classifier.classify(features) # classifiers' proposed polarity for tests
        testSets[predicted].add(i)

    #Outputs
    print('train on %s instances, test on %s instances'% (len(trainFeatures), len(testFeatures)))
    print('accuracy:', nltk.classify.util.accuracy(classifier, testFeatures))
    print('pos precision:', scores.precision(referenceSets['pos'], testSets['pos']))
    print('pos recall:', scores.recall(referenceSets['pos'], testSets['pos']))
    print('neg precision:', scores.precision(referenceSets['neg'], testSets['neg']))
    print('neg recall:', scores.recall(referenceSets['neg'], testSets['neg']))
    classifier.show_most_informative_features(10)

def make_full_dict(words):
    #print('Words:',words)
    #print(dict([(word, True) for word in words]))
    return dict([(word, True) for word in words])

print('using all words as features')
evaluate_features(make_full_dict)


using all words as features
train on 7998 instances, test on 2666 instances
accuracy: 0.77344336084021
pos precision: 0.7881422924901186
pos recall: 0.7479369842460615
neg precision: 0.7601713062098501
neg recall: 0.7989497374343586
Most Informative Features
              engrossing = True              pos : neg    =     17.0 : 1.0
                   quiet = True              pos : neg    =     15.7 : 1.0
                mediocre = True              neg : pos    =     13.7 : 1.0
               absorbing = True              pos : neg    =     13.0 : 1.0
                portrait = True              pos : neg    =     12.4 : 1.0
              refreshing = True              pos : neg    =     12.3 : 1.0
                   flaws = True              pos : neg    =     12.3 : 1.0
               inventive = True              pos : neg    =     12.3 : 1.0
            refreshingly = True              pos : neg    =     11.7 : 1.0
                 triumph = True              pos : neg    =     11

### 3a Stopwords removal

In [59]:
import re, math, collections, itertools, sys, os
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures, scores
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import re

def evaluate_features(feature_select):

    negSentences = open('rt-polarity-neg.txt', 'r', encoding='utf8')
    posSentences = open('rt-polarity-pos.txt', 'r', encoding='utf8')

    negSentences = re.split(r'\n', negSentences.read())
    posSentences = re.split(r'\n', posSentences.read())

    tweet_tk = TweetTokenizer() 
    swords = stopwords.words('english')
    tweets_pos = []
    tweets_neg = []

    for i in posSentences:
        tokenised_tweet=tweet_tk.tokenize(i)
        lower_tweet = [i.lower() for i in tokenised_tweet]
        for l in range(len(lower_tweet)):
            lower_tweet[l]= re.sub(r'[\d]', ' ', lower_tweet[l])
            lower_tweet[l]= re.sub(r'[^\w_]','',lower_tweet[l])
        tweets_pos.append([w for w in lower_tweet if w not in swords])

    final_pos = []
    for tweet in tweets_pos:
        tweet_cleaned = [a for a in tweet if a]
        " ".join(tweet_cleaned)
        final_pos.append(" ".join(tweet_cleaned))
        #final_pos.append(tweet_cleaned)

    #print('Positive:',final_pos[0])
    posSentences = final_pos
    #print(posSentences[0])

    for i in negSentences:
        tokenised_tweet=tweet_tk.tokenize(i)
        lower_tweet = [i.lower() for i in tokenised_tweet]
        for l in range(len(lower_tweet)):
            lower_tweet[l]= re.sub(r'[\d]', ' ', lower_tweet[l])
            lower_tweet[l]= re.sub(r'[^\w_]','',lower_tweet[l])
        tweets_neg.append([w for w in lower_tweet if w not in swords])

    final_neg = []
    for tweet in tweets_neg:
        tweet_cleaned = [a for a in tweet if a]
        final_neg.append(" ".join(tweet_cleaned))
    #print('\n\nNegative:',final_neg[0])
    negSentences = final_neg
    #print(negSentences[0])
    
    posFeatures = []
    negFeatures = []
    # breaks up the sentences into lists of individual words
    # creates instance structures for classifier
    
    for i in posSentences:
        posWords = re.findall(r"[\w']+|[.,!?;]", i)
        posWords = [feature_select(posWords), 'pos']
        posFeatures.append(posWords)
    for i in negSentences:
        negWords = re.findall(r"[\w']+|[.,!?;]", i)
        negWords = [feature_select(negWords), 'neg']
        negFeatures.append(negWords)
        
    posCutoff = int(math.floor(len(posFeatures)*3/4))
    negCutoff = int(math.floor(len(negFeatures)*3/4))
    trainFeatures = posFeatures[:posCutoff] + negFeatures[:negCutoff]
    testFeatures = posFeatures[posCutoff:] + negFeatures[negCutoff:]
    #Runs the classifier on the testFeatures
    classifier = NaiveBayesClassifier.train(trainFeatures)
    
    #Sets up labels to look at output
    referenceSets = collections.defaultdict(set)
    testSets = collections.defaultdict(set)
    for i, (features, label) in enumerate(testFeatures): # enumerate adds number-count to each item
        referenceSets[label].add(i)               # recorded polarity for these test sentences
        predicted = classifier.classify(features) # classifiers' proposed polarity for tests
        testSets[predicted].add(i)

    #Outputs
    print('train on %s instances, test on %s instances'% (len(trainFeatures), len(testFeatures)))
    print('accuracy:', nltk.classify.util.accuracy(classifier, testFeatures))
    print('pos precision:', scores.precision(referenceSets['pos'], testSets['pos']))
    print('pos recall:', scores.recall(referenceSets['pos'], testSets['pos']))
    print('neg precision:', scores.precision(referenceSets['neg'], testSets['neg']))
    print('neg recall:', scores.recall(referenceSets['neg'], testSets['neg']))
    #classifier.show_most_informative_features(10)
    
def make_full_dict(words):
    #print('Words:',words)
    #print(dict([(word, True) for word in words]))
    return dict([(word, True) for word in words])

print('After Stopword removal')
print('using all words as features')
evaluate_features(make_full_dict)    

After Stopword removal
using all words as features
train on 7998 instances, test on 2666 instances
accuracy: 0.7625656414103525
pos precision: 0.7611940298507462
pos recall: 0.7651912978244562
neg precision: 0.7639517345399698
neg recall: 0.759939984996249


## 3b Different Classifier: SVM

In [87]:
# http://andybromberg.com/sentiment-analysis-python/
# Andy Bromberg's Simple Sentiment Analysis System
# Uses data from Pang & Lee (2005)
# Uses a Naive Bayes Classifier Train the System
# NB Updated 2016 for package changes around scores

import re, math, collections, itertools, sys, os
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures, scores
from nltk.probability import FreqDist, ConditionalFreqDist
from sklearn.svm import LinearSVC

def evaluate_features(feature_select):
    #reading pre-labeled input and splitting into lines
    #Opens the positive,negative files.
    negSentences = open('rt-polarity-neg.txt', 'r', encoding='utf8')
    posSentences = open('rt-polarity-pos.txt', 'r', encoding='utf8')
    
    #Split the file content by newline character.
    negSentences = re.split(r'\n', negSentences.read())
    #print(negSentences)
    posSentences = re.split(r'\n', posSentences.read())
    #print(posSentences)
    
    posFeatures = []
    negFeatures = []
    # breaks up the sentences into lists of individual words
    # creates instance structures for classifier
    
    for i in posSentences:
        posWords = re.findall(r"[\w']+|[.,!?;]", i)
        posWords = [feature_select(posWords), 'pos']
        posFeatures.append(posWords)
    for i in negSentences:
        negWords = re.findall(r"[\w']+|[.,!?;]", i)
        negWords = [feature_select(negWords), 'neg']
        negFeatures.append(negWords)
        
    posCutoff = int(math.floor(len(posFeatures)*3/4))
    negCutoff = int(math.floor(len(negFeatures)*3/4))
    trainFeatures = posFeatures[:posCutoff] + negFeatures[:negCutoff]
    testFeatures = posFeatures[posCutoff:] + negFeatures[negCutoff:]
    #Runs the classifier on the testFeatures
    #classifier = NaiveBayesClassifier.train(trainFeatures)
    classifier = SklearnClassifier(LinearSVC())
    classifier.train(trainFeatures)
    
    #Sets up labels to look at output
    referenceSets = collections.defaultdict(set)
    testSets = collections.defaultdict(set)
    for i, (features, label) in enumerate(testFeatures): # enumerate adds number-count to each item
        referenceSets[label].add(i)               # recorded polarity for these test sentences
        predicted = classifier.classify(features) # classifiers' proposed polarity for tests
        testSets[predicted].add(i)

    #Outputs
    print('train on %s instances, test on %s instances'% (len(trainFeatures), len(testFeatures)))
    print('accuracy:', nltk.classify.util.accuracy(classifier, testFeatures))
    print('pos precision:', scores.precision(referenceSets['pos'], testSets['pos']))
    print('pos recall:', scores.recall(referenceSets['pos'], testSets['pos']))
    print('neg precision:', scores.precision(referenceSets['neg'], testSets['neg']))
    print('neg recall:', scores.recall(referenceSets['neg'], testSets['neg']))
    #classifier.show_most_informative_features(10)

def make_full_dict(words):
    #print('Words:',words)
    #print(dict([(word, True) for word in words]))
    return dict([(word, True) for word in words])

print('using all words as features')
evaluate_features(make_full_dict)


using all words as features
train on 7998 instances, test on 2666 instances
accuracy: 0.7475618904726181
pos precision: 0.7511415525114156
pos recall: 0.7404351087771943
neg precision: 0.7440828402366864
neg recall: 0.754688672168042


## 3b : Different Feature Selection

In [94]:
import re, math, collections, itertools, os
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures
from nltk import(precision, recall)
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.corpus import stopwords 

POLARITY_DATA_DIR = os.path.join('polarityData', 'rt-polaritydata')
RT_POLARITY_POS_FILE = os.path.join('rt-polarity-pos.txt')
RT_POLARITY_NEG_FILE = os.path.join('rt-polarity-neg.txt')


#this function takes a feature selection mechanism and returns its performance in a variety of metrics
def evaluate_features(feature_select):
    stop = set(stopwords.words('english'))
    posFeatures = []
    negFeatures = []
    #http://stackoverflow.com/questions/367155/splitting-a-string-into-words-and-punctuation
    #breaks up the sentences into lists of individual words (as selected by the input mechanism) and appends 'pos' or 'neg' after each list
    with open(RT_POLARITY_POS_FILE, 'r',encoding = 'utf-8') as posSentences:
        for i in posSentences:
            posWords = re.findall(r"[\w']+|[.,!?;]", i.rstrip())
            #posWords = [word for word in posWords if not word in stop]
            posWords = [feature_select(posWords), 'pos']
            posFeatures.append(posWords)
                    
    with open(RT_POLARITY_NEG_FILE, 'r',encoding = 'utf-8') as negSentences:
        for i in negSentences:
            negWords = re.findall(r"[\w']+|[.,!?;]", i.rstrip())
            #negWords = [word for word in negWords if not word in stop]
            negWords = [feature_select(negWords), 'neg']
            negFeatures.append(negWords)


    #selects 3/4 of the features to be used for training and 1/4 to be used for testing
    posCutoff = int(math.floor(len(posFeatures)*0.85))
    negCutoff = int(math.floor(len(negFeatures)*0.85))
    trainFeatures = posFeatures[:posCutoff] + negFeatures[:negCutoff]
    testFeatures = posFeatures[posCutoff:] + negFeatures[negCutoff:]

    #trains a Naive Bayes Classifier
    classifier = NaiveBayesClassifier.train(trainFeatures)	

    #initiates referenceSets and testSets
    referenceSets = collections.defaultdict(set)
    testSets = collections.defaultdict(set)	

    #puts correctly labeled sentences in referenceSets and the predictively labeled version in testsets
    for i, (features, label) in enumerate(testFeatures):
        referenceSets[label].add(i)
        predicted = classifier.classify(features)
        testSets[predicted].add(i)	

    #prints metrics to show how well the feature selection did
    print ('train on %d instances, test on %d instances' % (len(trainFeatures), len(testFeatures)))
    print ('accuracy:', nltk.classify.util.accuracy(classifier, testFeatures))
    print ('pos precision:', precision(referenceSets['pos'], testSets['pos']))
    print ('pos recall:', recall(referenceSets['pos'], testSets['pos']))
    print ('neg precision:', precision(referenceSets['neg'], testSets['neg']))
    print ('neg recall:', recall(referenceSets['neg'], testSets['neg']))
    classifier.show_most_informative_features(10)

#creates a feature selection mechanism that uses all words
def make_full_dict(words):
    return dict([(word, True) for word in words])

#tries using all words as the feature selection mechanism
print ('using all words as features')
evaluate_features(make_full_dict)

#scores words based on chi-squared test to show information gain (http://streamhacker.com/2010/06/16/text-classification-sentiment-analysis-eliminate-low-information-features/)
def create_word_scores():
    #creates lists of all positive and negative words
    posWords = []
    negWords = []
    with open(RT_POLARITY_POS_FILE, 'r',encoding = 'utf-8') as posSentences:
        for i in posSentences:
            posWord = re.findall(r"[\w']+|[.,!?;]", i.rstrip())
            posWords.append(posWord)
    with open(RT_POLARITY_NEG_FILE, 'r',encoding = 'utf-8') as negSentences:
        for i in negSentences:
            negWord = re.findall(r"[\w']+|[.,!?;]", i.rstrip())
            negWords.append(negWord)
    posWords = list(itertools.chain(*posWords))
    negWords = list(itertools.chain(*negWords))

    #build frequency distibution of all words and then frequency distributions of words within positive and negative labels
    word_fd = FreqDist()
    cond_word_fd = ConditionalFreqDist()
    for word in posWords:
        word_fd[word.lower()] += 1
        cond_word_fd['pos'][word.lower()] += 1
    for word in negWords:
        word_fd[word.lower()] += 1
        cond_word_fd['neg'][word.lower()] += 1

    #finds the number of positive and negative words, as well as the total number of words
    pos_word_count = cond_word_fd['pos'].N()
    neg_word_count = cond_word_fd['neg'].N()
    total_word_count = pos_word_count + neg_word_count

    #builds dictionary of word scores based on chi-squared test
    word_scores = {}
    for word, freq in word_fd.items():
        pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word], (freq, pos_word_count), total_word_count)
        neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word], (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score

    return word_scores

#finds word scores
word_scores = create_word_scores()

#finds the best 'number' words based on word scores
def find_best_words(word_scores, number):
    best_vals = sorted(word_scores.items(), key=lambda s: s, reverse=True)[:number]
    best_words = set([w for w, s in best_vals])
    return best_words

#creates feature selection mechanism that only uses best words
def best_word_features(words):
    return dict([(word, True) for word in words if word in best_words])

#numbers of features to select
numbers_to_test = [10, 100, 1000, 10000, 15000]
#tries the best_word_features mechanism with each of the numbers_to_test of features
for num in numbers_to_test:
    print ('evaluating best %d word features' % (num))
    best_words = find_best_words(word_scores, num)
    evaluate_features(best_word_features)

using all words as features
train on 9062 instances, test on 1600 instances
accuracy: 0.781875
pos precision: 0.7955439056356488
pos recall: 0.75875
neg precision: 0.7694145758661888
neg recall: 0.805
Most Informative Features
              engrossing = True              pos : neg    =     20.3 : 1.0
                 generic = True              neg : pos    =     14.3 : 1.0
                   flaws = True              pos : neg    =     14.3 : 1.0
              refreshing = True              pos : neg    =     13.7 : 1.0
                mediocre = True              neg : pos    =     13.7 : 1.0
               absorbing = True              pos : neg    =     13.0 : 1.0
               inventive = True              pos : neg    =     13.0 : 1.0
                  boring = True              neg : pos    =     12.7 : 1.0
                    flat = True              neg : pos    =     12.6 : 1.0
                 routine = True              neg : pos    =     12.3 : 1.0
evaluating best 10 word